# Notebook 2: Comparação de Modelos KMeans - Sinopse vs Todas as Features

**Objetivo:** Comparar o desempenho do modelo KMeans utilizando apenas sinopses vetorizadas versus utilizando todas as features disponíveis

**Desenvolvido por:** [Nome dos integrantes do grupo]

**Data:** 2025


## 1. Importação das Bibliotecas


In [ ]:
# Configurações para garantir que os prints funcionem
import sys
import os
sys.stdout.flush()

# Configurar matplotlib para funcionar em notebooks
import matplotlib
matplotlib.use('inline')

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px
from wordcloud import WordCloud
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.cluster import KMeans
from sklearn.metrics import silhouette_score, calinski_harabasz_score, davies_bouldin_score
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.decomposition import PCA
import warnings

# Configurações do pandas para melhor display
warnings.filterwarnings("ignore")
pd.set_option('display.max_rows', 1000)
pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 1000)
pd.set_option('display.max_colwidth', 100)

# Configurações do matplotlib
plt.rcParams['figure.figsize'] = (12, 8)
plt.rcParams['font.size'] = 10

# Funções de display melhoradas
import IPython
from IPython.display import display, HTML, clear_output

def print_and_display(text):
    """Função que faz print e também display para garantir visibilidade"""
    print(text)
    display(HTML(f"<div style='background-color: #f0f0f0; padding: 10px; border-left: 4px solid #007acc;'>{text}</div>"))

def display_df(df, title="DataFrame"):
    """Função para display de DataFrames com título"""
    display(HTML(f"<h4>{title}</h4>"))
    display(df)

print_and_display("✅ Bibliotecas importadas com sucesso!")
print_and_display(f"Python version: {sys.version}")
print_and_display(f"Pandas version: {pd.__version__}")
print_and_display("=" * 50)


## 2. Carregamento e Preparação dos Dados


In [ ]:
# Carregar dataset (use o arquivo gerado pelo Notebook 1 ou o dataset original)
try:
    # Tentar carregar o dataset com clusters do Notebook 1
    df = pd.read_csv('imdb_top250_with_clusters.csv', sep=';')
    print("Dataset com clusters carregado com sucesso!")
except:
    try:
        # Tentar carregar o dataset original
        df = pd.read_csv('all_movies.csv', sep=';')
        print("Dataset original carregado com sucesso!")
    except:
        print("Erro ao carregar dataset. Verifique se os arquivos existem.")
        # Criar dataset de exemplo para demonstração
        df = pd.DataFrame({
            'title_pt': ['Filme 1', 'Filme 2', 'Filme 3'],
            'title_en': ['Movie 1', 'Movie 2', 'Movie 3'],
            'year': [2020, 2021, 2022],
            'rating': [8.5, 7.8, 9.2],
            'genre': ['Action', 'Drama', 'Comedy'],
            'sinopse': ['Ação emocionante', 'Drama tocante', 'Comédia divertida']
        })

print(f"Shape do dataset: {df.shape}")
print(f"Colunas: {list(df.columns)}")
print("\nPrimeiras linhas:")
print(df.head())


In [ ]:
# Preparação dos dados
df_clean = df.copy()

# Converter tipos de dados
df_clean['year'] = pd.to_numeric(df_clean['year'], errors='coerce')
df_clean['rating'] = pd.to_numeric(df_clean['rating'], errors='coerce')

# Remover linhas com valores críticos nulos
df_clean = df_clean.dropna(subset=['title_en', 'sinopse'])

# Preencher valores nulos
df_clean['genre'] = df_clean['genre'].fillna('Unknown')

# Limpeza de texto (se não foi feita no Notebook 1)
if 'sinopse_clean' not in df_clean.columns:
    import re
    import nltk
    from nltk.corpus import stopwords
    
    try:
        nltk.download('stopwords', quiet=True)
        stop_words = set(stopwords.words('english'))
    except:
        stop_words = {'the', 'a', 'an', 'and', 'or', 'but', 'in', 'on', 'at', 'to', 'for', 'of', 'with', 'by'}
    
    def clean_text(text):
        if pd.isna(text) or text == 'N/A':
            return ""
        text = str(text).lower()
        text = re.sub(r'[^a-zA-Z\s]', '', text)
        words = text.split()
        words = [word for word in words if word not in stop_words and len(word) > 2]
        return ' '.join(words)
    
    df_clean['sinopse_clean'] = df_clean['sinopse'].apply(clean_text)

# Remover filmes sem sinopse válida
df_clean = df_clean[df_clean['sinopse_clean'].str.len() > 10]

print(f"Dataset após limpeza: {df_clean.shape}")
print(f"Valores nulos restantes:")
print(df_clean.isnull().sum())


## 3. Modelo 1: KMeans com Apenas Sinopses Vetorizadas


In [ ]:
# Modelo 1: Apenas sinopses vetorizadas
print("=== MODELO 1: APENAS SINOPSES VETORIZADAS ===\n")

# Aplicar TF-IDF
vectorizer = TfidfVectorizer(
    max_features=1000,
    min_df=2,
    max_df=0.8,
    ngram_range=(1, 2)
)

X_tfidf = vectorizer.fit_transform(df_clean['sinopse_clean'])
print(f"Shape da matriz TF-IDF: {X_tfidf.shape}")

# Treinar KMeans
kmeans_model1 = KMeans(
    n_clusters=5,
    random_state=42,
    n_init=10,
    max_iter=300
)

cluster_labels_model1 = kmeans_model1.fit_predict(X_tfidf)
df_clean['cluster_model1'] = cluster_labels_model1

# Calcular métricas
silhouette_model1 = silhouette_score(X_tfidf, cluster_labels_model1)
calinski_model1 = calinski_harabasz_score(X_tfidf.toarray(), cluster_labels_model1)
davies_bouldin_model1 = davies_bouldin_score(X_tfidf.toarray(), cluster_labels_model1)

print(f"Distribuição dos clusters (Modelo 1):")
print(df_clean['cluster_model1'].value_counts().sort_index())
print(f"\nMétricas do Modelo 1:")
print(f"Silhouette Score: {silhouette_model1:.3f}")
print(f"Calinski-Harabasz Score: {calinski_model1:.3f}")
print(f"Davies-Bouldin Score: {davies_bouldin_model1:.3f}")


## 4. Modelo 2: KMeans com Todas as Features


In [ ]:
# Modelo 2: Todas as features
print("=== MODELO 2: TODAS AS FEATURES ===\n")

# Preparar features numéricas
df_features = df_clean.copy()

# Codificar gênero
le_genre = LabelEncoder()
df_features['genre_encoded'] = le_genre.fit_transform(df_features['genre'])

# Criar features adicionais
df_features['word_count'] = df_features['sinopse_clean'].apply(lambda x: len(x.split()))
df_features['char_count'] = df_features['sinopse_clean'].apply(lambda x: len(x))

# Normalizar features numéricas
scaler = StandardScaler()
numeric_features = ['year', 'rating', 'genre_encoded', 'word_count', 'char_count']
df_features[numeric_features] = scaler.fit_transform(df_features[numeric_features])

# Combinar features numéricas com TF-IDF
X_numeric = df_features[numeric_features].values
X_combined = np.hstack([X_numeric, X_tfidf.toarray()])

print(f"Shape das features numéricas: {X_numeric.shape}")
print(f"Shape das features combinadas: {X_combined.shape}")

# Treinar KMeans
kmeans_model2 = KMeans(
    n_clusters=5,
    random_state=42,
    n_init=10,
    max_iter=300
)

cluster_labels_model2 = kmeans_model2.fit_predict(X_combined)
df_clean['cluster_model2'] = cluster_labels_model2

# Calcular métricas
silhouette_model2 = silhouette_score(X_combined, cluster_labels_model2)
calinski_model2 = calinski_harabasz_score(X_combined, cluster_labels_model2)
davies_bouldin_model2 = davies_bouldin_score(X_combined, cluster_labels_model2)

print(f"Distribuição dos clusters (Modelo 2):")
print(df_clean['cluster_model2'].value_counts().sort_index())
print(f"\nMétricas do Modelo 2:")
print(f"Silhouette Score: {silhouette_model2:.3f}")
print(f"Calinski-Harabasz Score: {calinski_model2:.3f}")
print(f"Davies-Bouldin Score: {davies_bouldin_model2:.3f}")


## 5. Comparação dos Modelos


In [ ]:
# Comparação das métricas
print("=== COMPARAÇÃO DAS MÉTRICAS ===\n")

comparison_data = {
    'Métrica': ['Silhouette Score', 'Calinski-Harabasz Score', 'Davies-Bouldin Score'],
    'Modelo 1 (Apenas Sinopse)': [silhouette_model1, calinski_model1, davies_bouldin_model1],
    'Modelo 2 (Todas Features)': [silhouette_model2, calinski_model2, davies_bouldin_model2]
}

comparison_df = pd.DataFrame(comparison_data)
print(comparison_df.round(3))

# Visualização da comparação
fig, axes = plt.subplots(1, 3, figsize=(15, 5))

metrics = ['Silhouette Score', 'Calinski-Harabasz Score', 'Davies-Bouldin Score']
model1_values = [silhouette_model1, calinski_model1, davies_bouldin_model1]
model2_values = [silhouette_model2, calinski_model2, davies_bouldin_model2]

for i, (metric, val1, val2) in enumerate(zip(metrics, model1_values, model2_values)):
    axes[i].bar(['Modelo 1', 'Modelo 2'], [val1, val2], color=['#FF6B6B', '#4ECDC4'])
    axes[i].set_title(metric)
    axes[i].set_ylabel('Valor')
    
    # Adicionar valores nas barras
    axes[i].text(0, val1 + max(val1, val2) * 0.01, f'{val1:.3f}', ha='center', va='bottom')
    axes[i].text(1, val2 + max(val1, val2) * 0.01, f'{val2:.3f}', ha='center', va='bottom')

plt.tight_layout()
plt.show()

# Determinar qual modelo é melhor para cada métrica
print("\n=== ANÁLISE DOS RESULTADOS ===")
print("Silhouette Score (maior é melhor):")
if silhouette_model1 > silhouette_model2:
    print(f"  Modelo 1 é melhor: {silhouette_model1:.3f} vs {silhouette_model2:.3f}")
else:
    print(f"  Modelo 2 é melhor: {silhouette_model2:.3f} vs {silhouette_model1:.3f}")

print("\nCalinski-Harabasz Score (maior é melhor):")
if calinski_model1 > calinski_model2:
    print(f"  Modelo 1 é melhor: {calinski_model1:.3f} vs {calinski_model2:.3f}")
else:
    print(f"  Modelo 2 é melhor: {calinski_model2:.3f} vs {calinski_model1:.3f}")

print("\nDavies-Bouldin Score (menor é melhor):")
if davies_bouldin_model1 < davies_bouldin_model2:
    print(f"  Modelo 1 é melhor: {davies_bouldin_model1:.3f} vs {davies_bouldin_model2:.3f}")
else:
    print(f"  Modelo 2 é melhor: {davies_bouldin_model2:.3f} vs {davies_bouldin_model1:.3f}")


In [ ]:
# Análise detalhada dos clusters de cada modelo
print("=== ANÁLISE DETALHADA DOS CLUSTERS ===\n")

# Função para analisar clusters
def analyze_clusters(df, cluster_col, model_name):
    print(f"--- {model_name} ---")
    for cluster_id in sorted(df[cluster_col].unique()):
        cluster_data = df[df[cluster_col] == cluster_id]
        
        print(f"\nCluster {cluster_id}:")
        print(f"  Número de filmes: {len(cluster_data)}")
        print(f"  Rating médio: {cluster_data['rating'].mean():.2f}")
        print(f"  Ano médio: {cluster_data['year'].mean():.0f}")
        
        # Gêneros mais comuns
        top_genres = cluster_data['genre'].value_counts().head(3)
        print(f"  Gêneros principais: {', '.join([f'{genre} ({count})' for genre, count in top_genres.items()])}")
        
        # Filmes mais bem avaliados
        top_movies = cluster_data.nlargest(2, 'rating')[['title_en', 'rating']]
        print(f"  Filmes top: {', '.join([f'{movie[0]} ({movie[1]:.1f})' for _, movie in top_movies.iterrows()])}")

# Analisar clusters do Modelo 1
analyze_clusters(df_clean, 'cluster_model1', 'MODELO 1 (Apenas Sinopse)')

print("\n" + "="*50 + "\n")

# Analisar clusters do Modelo 2
analyze_clusters(df_clean, 'cluster_model2', 'MODELO 2 (Todas Features)')


## 6. Visualizações Comparativas


In [ ]:
# Visualizações comparativas
fig, axes = plt.subplots(2, 3, figsize=(18, 12))

# 1. Distribuição dos clusters - Modelo 1
axes[0, 0].bar(df_clean['cluster_model1'].value_counts().sort_index().index, 
               df_clean['cluster_model1'].value_counts().sort_index().values,
               color=['#FF6B6B', '#4ECDC4', '#45B7D1', '#96CEB4', '#FFEAA7'])
axes[0, 0].set_title('Modelo 1: Distribuição dos Clusters')
axes[0, 0].set_xlabel('Cluster')
axes[0, 0].set_ylabel('Número de Filmes')

# 2. Distribuição dos clusters - Modelo 2
axes[0, 1].bar(df_clean['cluster_model2'].value_counts().sort_index().index, 
               df_clean['cluster_model2'].value_counts().sort_index().values,
               color=['#FF6B6B', '#4ECDC4', '#45B7D1', '#96CEB4', '#FFEAA7'])
axes[0, 1].set_title('Modelo 2: Distribuição dos Clusters')
axes[0, 1].set_xlabel('Cluster')
axes[0, 1].set_ylabel('Número de Filmes')

# 3. Rating médio por cluster - Comparação
rating_model1 = df_clean.groupby('cluster_model1')['rating'].mean()
rating_model2 = df_clean.groupby('cluster_model2')['rating'].mean()

x = np.arange(len(rating_model1))
width = 0.35

axes[0, 2].bar(x - width/2, rating_model1.values, width, label='Modelo 1', color='#FF6B6B')
axes[0, 2].bar(x + width/2, rating_model2.values, width, label='Modelo 2', color='#4ECDC4')
axes[0, 2].set_title('Rating Médio por Cluster')
axes[0, 2].set_xlabel('Cluster')
axes[0, 2].set_ylabel('Rating Médio')
axes[0, 2].set_xticks(x)
axes[0, 2].legend()

# 4. Ano médio por cluster - Comparação
year_model1 = df_clean.groupby('cluster_model1')['year'].mean()
year_model2 = df_clean.groupby('cluster_model2')['year'].mean()

axes[1, 0].bar(x - width/2, year_model1.values, width, label='Modelo 1', color='#FF6B6B')
axes[1, 0].bar(x + width/2, year_model2.values, width, label='Modelo 2', color='#4ECDC4')
axes[1, 0].set_title('Ano Médio por Cluster')
axes[1, 0].set_xlabel('Cluster')
axes[1, 0].set_ylabel('Ano Médio')
axes[1, 0].set_xticks(x)
axes[1, 0].legend()

# 5. Gêneros por cluster - Modelo 1
genre_cluster1 = pd.crosstab(df_clean['genre'], df_clean['cluster_model1'])
genre_cluster1_pct = genre_cluster1.div(genre_cluster1.sum(axis=0), axis=1) * 100
top_genres = df_clean['genre'].value_counts().head(5).index
genre_cluster1_pct.loc[top_genres].plot(kind='bar', ax=axes[1, 1])
axes[1, 1].set_title('Modelo 1: Gêneros por Cluster')
axes[1, 1].set_xlabel('Gênero')
axes[1, 1].set_ylabel('Percentual (%)')
axes[1, 1].tick_params(axis='x', rotation=45)

# 6. Gêneros por cluster - Modelo 2
genre_cluster2 = pd.crosstab(df_clean['genre'], df_clean['cluster_model2'])
genre_cluster2_pct = genre_cluster2.div(genre_cluster2.sum(axis=0), axis=1) * 100
genre_cluster2_pct.loc[top_genres].plot(kind='bar', ax=axes[1, 2])
axes[1, 2].set_title('Modelo 2: Gêneros por Cluster')
axes[1, 2].set_xlabel('Gênero')
axes[1, 2].set_ylabel('Percentual (%)')
axes[1, 2].tick_params(axis='x', rotation=45)

plt.tight_layout()
plt.show()


## 7. Escolha do Melhor Modelo e Justificativa


In [ ]:
# Análise final e escolha do melhor modelo
print("=== ANÁLISE FINAL E ESCOLHA DO MELHOR MODELO ===\n")

# Calcular score composto (média normalizada das métricas)
def calculate_composite_score(silhouette, calinski, davies_bouldin):
    """
    Calcula um score composto normalizando as métricas
    Silhouette e Calinski-Harabasz: maior é melhor
    Davies-Bouldin: menor é melhor
    """
    # Normalizar Silhouette (0-1)
    silhouette_norm = max(0, silhouette)  # Silhouette pode ser negativo
    
    # Normalizar Calinski-Harabasz (assumindo range 0-1000)
    calinski_norm = min(1, calinski / 1000)
    
    # Normalizar Davies-Bouldin (inverter, menor é melhor)
    davies_bouldin_norm = max(0, 1 - davies_bouldin / 5)  # Assumindo range 0-5
    
    # Score composto (média ponderada)
    composite_score = (silhouette_norm * 0.4 + calinski_norm * 0.3 + davies_bouldin_norm * 0.3)
    return composite_score

score_model1 = calculate_composite_score(silhouette_model1, calinski_model1, davies_bouldin_model1)
score_model2 = calculate_composite_score(silhouette_model2, calinski_model2, davies_bouldin_model2)

print(f"Score Composto Modelo 1: {score_model1:.3f}")
print(f"Score Composto Modelo 2: {score_model2:.3f}")

# Determinar o melhor modelo
if score_model1 > score_model2:
    best_model = "Modelo 1 (Apenas Sinopse)"
    best_score = score_model1
    print(f"\n🏆 MELHOR MODELO: {best_model}")
else:
    best_model = "Modelo 2 (Todas Features)"
    best_score = score_model2
    print(f"\n🏆 MELHOR MODELO: {best_model}")

print(f"Score: {best_score:.3f}")

# Análise de consistência dos clusters
print(f"\n=== ANÁLISE DE CONSISTÊNCIA ===")

# Verificar se os clusters são consistentes entre os modelos
from sklearn.metrics import adjusted_rand_score

ari_score = adjusted_rand_score(df_clean['cluster_model1'], df_clean['cluster_model2'])
print(f"Adjusted Rand Index (consistência entre modelos): {ari_score:.3f}")

if ari_score > 0.5:
    print("Os modelos produzem clusters relativamente consistentes")
elif ari_score > 0.2:
    print("Os modelos produzem clusters parcialmente consistentes")
else:
    print("Os modelos produzem clusters muito diferentes")


### 7.1 Justificativa da Escolha

**Critérios de Avaliação:**

1. **Métricas Quantitativas:**
   - **Silhouette Score:** Mede a qualidade da separação dos clusters (maior é melhor)
   - **Calinski-Harabasz Score:** Mede a razão entre dispersão entre clusters e dentro dos clusters (maior é melhor)
   - **Davies-Bouldin Score:** Mede a qualidade da separação baseada na distância intra-cluster (menor é melhor)

2. **Interpretabilidade:**
   - Capacidade de interpretar e explicar os clusters encontrados
   - Coerência com características conhecidas dos filmes

3. **Aplicabilidade Prática:**
   - Facilidade de implementação em sistemas de recomendação
   - Robustez para novos dados

**Justificativa da Escolha:**

[Baseado nos resultados obtidos, justificar qual modelo foi escolhido e por quê]

**Vantagens do Modelo Escolhido:**
- [Listar vantagens específicas]

**Limitações do Modelo Escolhido:**
- [Listar limitações e como contorná-las]

**Recomendações para Implementação:**
- [Sugerir melhorias e próximos passos]


In [ ]:
# Salvar resultados da comparação
print("Salvando resultados da comparação...")

# Salvar DataFrame com ambos os modelos
df_clean.to_csv('imdb_comparison_results.csv', index=False, sep=';')
print("Resultados salvos em 'imdb_comparison_results.csv'")

# Criar resumo da comparação
comparison_summary = {
    'Modelo': ['Modelo 1 (Apenas Sinopse)', 'Modelo 2 (Todas Features)'],
    'Silhouette_Score': [silhouette_model1, silhouette_model2],
    'Calinski_Harabasz_Score': [calinski_model1, calinski_model2],
    'Davies_Bouldin_Score': [davies_bouldin_model1, davies_bouldin_model2],
    'Composite_Score': [score_model1, score_model2],
    'Melhor_Modelo': [best_model == 'Modelo 1 (Apenas Sinopse)', best_model == 'Modelo 2 (Todas Features)']
}

comparison_summary_df = pd.DataFrame(comparison_summary)
comparison_summary_df.to_csv('model_comparison_summary.csv', index=False, sep=';')
print("Resumo da comparação salvo em 'model_comparison_summary.csv'")

print("\n=== RESUMO FINAL ===")
print(f"Total de filmes analisados: {len(df_clean)}")
print(f"Modelo escolhido: {best_model}")
print(f"Score composto: {best_score:.3f}")
print(f"Consistência entre modelos (ARI): {ari_score:.3f}")

print("\nMétricas finais:")
print(comparison_summary_df.round(3))
